In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-09-07T14:47:48.839075-07:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.14.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-1029-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 256
Architecture: 64bit

